# MBQC 模型下求解多项式组合优化问题

<em> Copyright (c) 2021 Institute for Quantum Computing, Baidu Inc. All Rights Reserved. </em>

在[基于测量的量子近似优化算法](QAOA_CN.ipynb)中，我们介绍了多项式无约束布尔优化问题（polynomial unconstrained boolean optimization problem, PUBO），并提出了**基于测量的量子近似优化算法（MB-QAOA）** 对该类问题的求解方式，感兴趣的读者可以参阅前面的内容。这里我们给出两个示例作为 MB-QAOA 的实战演示：一个具体的 PUBO 问题和一个最大割问题。

## 示例：PUBO 问题

我们首先简单回顾一下什么是 PUBO 问题。给定一个变量为 $x = \{x_1,\cdots,x_n\}$ 的 $n$ 元多项式，

$$
C(x) = \sum_{\lambda \in \Lambda } \alpha_{\lambda} \prod_{j \in \lambda} x_j,\tag{1}
$$

其中每个变量 $x_i \in \{0,1\}$，$\prod_{j \in \lambda} x_j$ 为一个单项式，$\lambda \subseteq [n]:= \{1, 2, ..., n\}$ 为一个指标集，$\Lambda$ 为指标集的集合，$\alpha_\lambda$ 为每个单项式对应的实系数。在 PUBO 问题中，$C(x)$ 称为目标多项式。PUBO 问题就是寻找一组最优解 $x^* = \{x_1^*, x_2^*, ..., x_n^*\} $ 使得目标多项式的取值最大，即

$$
x^* =\underset{x}{\text{argmax}} \ C(x).\tag{2}
$$

代码实现上，我们规定输入多项式的标准形式为一个列表，列表中的第一个元素为多项式的最大变量数，列表的第二个元素为一个字典，字典的键为每个单项式涉及的变量（常数则为 ‘cons’），变量之间用逗号隔开，值为该单项式的系数。比如，我们希望输入一个三元多项式 $x_1 + x_2 - x_3 + x_1 x_2 - x_1 x_2 x_3 + 0.5$，应输入如下代码：

In [ ]:
# 变量数
var_num = 3
# 以字典格式存储多项式的各个单项式
poly_dict = {'x_1': 1, 'x_2': 1, 'x_3': -1, 'x_1,x_2': 1, 'x_1,x_2,x_3': -1, 'cons':0.5}
# 构造符合约定的多项式
polynomial = [var_num, poly_dict]

**注意：** 由于我们考虑的变量为布尔值，所以每个单项式中变量指数最大为 1，即输入的多项式字典的键中，每个变量最多出现一次。例如：{'x_1,x_1,x_2': 1} 为不符合规范的输入。我们同样约定自变量的下角标从 "1" 开始，与数学习惯相一致。并且为了统一输入规范，自变量需要连续编号，不能出现输入多项式为 $x_1x_2 + x_6$ 的情况，应该改为 $x_1x_2 + x_3$，否则我们会以报错的形式提醒输入不符合规范。

代码实现上，我们在 `pubo` 中定义了 `is_poly_valid` 函数，用于检查输入多项式的合法性。如果合法，将会打印 "The polynomial is valid." 字样，否则会报错。

```python
from paddle_quantum.mbqc.QAOA.pubo import is_poly_valid
```
我们同样定义了一个函数 `random_poly` 根据变量个数随机生成一个布尔型多项式。
```python
from paddle_quantum.mbqc.QAOA.pubo import random_poly
```
**注意：** 随机生成的多项式并不一定是合法的！因此我们需要在计算之前检查多项式的合法性。

### 求解 PUBO 问题的代码实现

In [ ]:
# 引入记时模块
from time import perf_counter
# 引入符号计算模块
from sympy import symbols
#引入 paddle 相关模块
from paddle import seed, optimizer
# 引入 pubo 模块
from paddle_quantum.mbqc.QAOA.pubo import dict_to_symbol, is_poly_valid, brute_force_search
# 引入 qaoa 模块 
from paddle_quantum.mbqc.QAOA.qaoa import MBQC_QAOA_Net, get_solution_string

利用 MB-QAOA 算法，我们可以定义 PUBO 主函数 ``mbqc_pubo``，输入目标函数和 QAOA 算法深度，最终返回最优解对应的比特串。在主函数中，**最核心的是 MBQC_QAOA_Net 类**，其集成了 MB-QAOA 以及优化网络，感兴趣的读者可以返回到 [基于测量的量子近似优化算法](QAOA_CN.ipynb) 中查看，这里我们直接调用。

In [ ]:
# 定义 MBQC 模型求解 PUBO 问题的主函数
def mbqc_pubo(OBJ_POLY, DEPTH, SEED, LR, ITR, EPOCH, SHOTS=1024):
    
    # 将字典类型的目标多项式转化为 sympy 能处理的符号多项式
    obj_poly = dict_to_symbol(OBJ_POLY)
    var_num, poly_symbol = obj_poly

    # 打印当前 QAOA 算法电路深度
    print("QAOA 算法深度为：", DEPTH)

    # 开始计时器
    start_time = perf_counter()
    
    # 初始化一个 MB-QAOA 训练网络
    seed(SEED)
    mbqc_net = MBQC_QAOA_Net(DEPTH)
    
    # 选择 Adams 优化器
    opt = optimizer.Adam(learning_rate=LR, parameters=mbqc_net.parameters())

    # 开始训练
    for epoch in range(EPOCH):
        # 每次迭代后更新训练参数
        for itr in range(1, ITR + 1):
            # 用网络进行训练并返回损失函数值和当前输出量子态
            loss, vector_out = mbqc_net(poly=obj_poly)
            # 根据损失函数值优化参数
            loss.backward()
            opt.minimize(loss)
            opt.clear_grad()
            if itr % 10 == 0:
                print("iter:", itr, "  loss_MBQC:", "%.4f" % loss.numpy())
                
    # 停止计时器并打印训练用时
    end_time = perf_counter()
    print("MBQC 运行时间为： ", end_time - start_time)

    # 打印优化后的训练参数 gamma, beta
    print("最优参数 gamma 为： ", mbqc_net.gamma.numpy())
    print("最优参数 beta 为： ", mbqc_net.beta.numpy())

    # 解码原问题的答案
    solution_str = get_solution_string(vector_out, SHOTS)

    # 将最优解带回目标函数，求出对应的最优值
    relation = {symbols('x_' + str(j + 1)): int(solution_str[j]) for j in range(var_num)}
    value = poly_symbol.evalf(subs=relation)
    
    # 返回最优解和最优值
    opt = [solution_str, value]
    return opt

为了检验训练结果的正确性，我们在 `pubo` 中定义了 `brute_force_search` 函数，用遍历的方法在解空间里暴力搜索 PUBO 问题的解，方便与 MBQC 模型训练得到的解进行对比。

```python
from paddle_quantum.mbqc.QAOA.pubo import brute_force_search
```

### Main 函数

主函数定义后，我们就可以输入参数运行啦！

In [ ]:
# 定义主函数
def main():
    
    # 以上面的例子 x_1 + x_2 - x_3 + x_1*x_2 -x_1*x_2*x_3 + 0.5 为例，求解 PUBO 问题
    var_num = 3  
    poly_dict = {'x_1': 1, 'x_2': 1, 'x_3': -1, 'x_1,x_2': 1, 'x_1,x_2,x_3': -1, 'cons':0.5}
    polynomial = [var_num, poly_dict]
    
    # 打印输入的多项式
    print("输入的多项式为：", polynomial)
    
    # 我们也可以随机生成一个目标多项式
    # polynomial = random_poly(var_num)

    # 检查输入多项式的合法性
    is_poly_valid(polynomial)

    # 进行训练并返回最优结果
    mbqc_result = mbqc_pubo(
        OBJ_POLY=polynomial,  # 目标多项式函数
        DEPTH=6,  # QAOA 算法电路深度
        SEED=1024,  # 随机种子
        LR=0.1,  # 学习率
        ITR=120,  # 训练迭代次数
        EPOCH=1  # 迭代周期
    )

    # 打印 MBQC 模型训练出来的最优结果
    print("MBQC 模型得到的最优解为：", mbqc_result[0])
    print("MBQC 模型得到的最优值为：", mbqc_result[1])
    
    # 通过暴力搜索计算 PUBO 问题的最优结果并打印
    brute_result = brute_force_search(polynomial)
    print("暴力搜索得到的最优解为：", brute_result[0])
    print("暴力搜索得到的最优值为：", brute_result[1])
    
    # 将两种结果进行比较
    print("MBQC 模型和暴力搜索的差值为：", mbqc_result[1] - brute_result[1])


if __name__ == '__main__':
    main()

## 示例：最大割问题

### 图与割

最大割问题（MaxCut Problem）是图论中常见的一个组合优化问题，在统计物理学和电路设计中都有重要应用。

在图论中，一个图由 $G = (V, E)$ 表示，其中 $V$ 为图的点集合，$E$ 为边集合。例如一个正方形就可以由 $G = (V,E)$ 其中 $V = [1,2,3,4]$， $E = [(1,2),(2,3),(3,4),(1,4)]$ 来表示。

代码上，我们可以用 `maxcut` 中的 `plot_graph` 画出给定的图。

In [ ]:
from paddle_quantum.mbqc.QAOA.maxcut import plot_graph
V = [1,2,3,4]
E = [(1,2),(2,3),(3,4),(1,4)]
G = [V, E] 
plot_graph(G,"A square")

一个图的割是指将该图的顶点集 $V$ 分割成两个互不相交的子集合 $S_0$ 和 $S_1$ 的一种划分。如果图中边的两个顶点被划分到不同集合中，那么记录得一分，总得分则定义为这个割的大小。最大割问题就是要找到一个割使得该割的大小最大。

比如对应如上所述正方形图 $G$， 其中一个最大割就是将点 $1$ 和 $3$ 分为同一个集合 $S_0$ 中，点 $2$ 和 $4$ 分到另一个集合 $S_1$ 中。  

### 转化为 PUBO 问题

最大割问题其实可以转化为一个 PUBO 问题。假设待分割的图 $G = (V, E)$ 有 $n=|V|$ 个顶点和 $m =|E|$ 条边，那么我们可以将最大割问题等价为拥有 $n$ 个自变量的组合优化问题。每个变量 $x_v$ 对应图 $G$ 中的一个顶点 $v \in V$，其取值 $x_v \in \{0,1\}$，分别对应于顶点属于集合 $S_0$ 或 $S_1$，因此 $x = \{x_1,\cdots,x_n\}$ 的每一种取值方式都对应一个割。由于最大割问题中有效计数的边为那些顶点属于不同集合的边，即顶点 $u$ 和 $v$ 对应的变量取值不同 $x_u \neq x_v$ 才会被记一分。所以对于给定的割 $x$，其对应的割的大小为

$$
C(x) = \sum_{(u,v) \in E} (x_u \oplus x_v),\tag{3}
$$

其中 $\oplus$ 为异或运算。那么最大割问题等价于求解 $ \underset{x}{\max} \ C(x)$。进一步，上述函数 $C(x)$ 可以写成多项式形式

$$
C(x) = \sum_{(u, v) \in E} (x_u + x_v - 2 x_u x_v),\tag{4}
$$

从而最大割问题等价于一个 $n$ 元 $2$ 次多项式的 PUBO 问题。我们希望找到最优解 $x^{*}$ 使得目标多项式最大，

$$
x^* = \underset{x}{\text{argmax}} \ \left( \sum_{(u, v) \in E} (x_u + x_v - 2 x_u x_v) \right).\tag{5}
$$

在 `maxcut` 当中，我们定义了 `graph_to_poly` 函数，可以输入待分割的图，返回等价的 PUBO 问题的目标多项式。

In [ ]:
# 引入 maxcut 模块
from paddle_quantum.mbqc.QAOA.maxcut import graph_to_poly

# 输入点和边的信息
V = [1,2,3,4]
E = [(1,2),(2,3),(3,4),(1,4)]
# 构造待分割的图
G = [V, E] 

# 将图转化为对应 PUBO 问题的目标多项式
poly = graph_to_poly(G)
print("与图等价的目标多项式为：\n", poly)

### 求解最大割问题的代码实现

得到等价的目标多项式之后，我们便可以仿照 PUBO 的流程，把最大割问题作为 PUBO 问题来求解。具体代码如下：

In [ ]:
# 引入符号运算模块
from sympy import symbols
# 引入 paddle 模块
from paddle import seed, optimizer
# 引入 qaoa 模块
from paddle_quantum.mbqc.QAOA.qaoa import MBQC_QAOA_Net, get_solution_string
# 引入 maxcut 模块
from paddle_quantum.mbqc.QAOA. maxcut import plot_graph, graph_to_poly, plot_solution

利用 MB-QAOA 算法，我们可以定义 MaxCut 主函数，将图输入到 MB-QAOA 算法中，最终返回求得的解并画出割的方式。

In [ ]:
# 定义 MaxCut 主函数
def mbqc_maxcut(GRAPH, DEPTH, SEED, LR, ITR, EPOCH, SHOTS=1024):
    
    # 打印待分割的图
    plot_graph(graph=GRAPH, title="Graph to be cut")

    # 找到图对应的目标多项式
    polynomial = graph_to_poly(GRAPH)
    print("与图等价的目标多项式为：", polynomial[1])

    # 开始算法的记时
    start_time = perf_counter()

    # 实例化一个 MB-QAOA 训练网络
    seed(SEED)
    mbqc_net = MBQC_QAOA_Net(DEPTH)
    # 选择 Adams 优化器
    opt = optimizer.Adam(learning_rate=LR, parameters=mbqc_net.parameters())

    # 开始训练
    for epoch in range(EPOCH):
        # 每次迭代后更新训练参数
        for itr in range(1, ITR + 1):
            # 训练参数并返回损失函数值和当前输出量子态
            loss, state_out = mbqc_net(poly=polynomial)
            # 根据损失函数值优化训练参数
            loss.backward()
            opt.minimize(loss)
            opt.clear_grad()
            if itr % 10 == 0:
                print("iter:", itr, "  loss_MBQC:", "%.4f" % loss.numpy())

    # 停止算法的记时并打印用时
    end_time = perf_counter()
    print("MBQC 运行时间为：", end_time - start_time)
    
    # 打印训练得到的最优参数
    print("最优参数 gamma 为： ", mbqc_net.gamma.numpy())
    print("最优参数 beta 为：", mbqc_net.beta.numpy())

    # 从量子态中解码问题的答案
    mbqc_solution = get_solution_string(state_out, SHOTS)
    # 绘制割的方式
    plot_solution(GRAPH, mbqc_solution)

    # 将最优解带入目标函数，求得对应最优值，并返回
    var_num, poly_symbol = polynomial
    relation = {symbols('x_' + str(j + 1)): int(mbqc_solution[j]) for j in range(var_num)}
    
    mbqc_value = int(poly_symbol.evalf(subs=relation))
    mbqc_opt = [mbqc_solution, mbqc_value]

    # 返回最优解和对应的最优值
    return mbqc_opt

### Main 函数

主函数定义后，我们就可以输入参数运行啦！

In [ ]:
def main():
    # 以正方形的 MaxCut 问题为例，输入节点和边信息
    V = [1, 2, 3, 4]
    E = [(1, 2), (2, 3), (3, 4), (4, 1)]
    G = [V, E]
    
    # 进行训练并返回最优结果
    mbqc_result = mbqc_maxcut(
        GRAPH=G,  #  待分割的图
        DEPTH=6,  # QAOA 算法电路深度
        SEED=1024,  # 随机种子
        LR=0.1,  # 学习率
        ITR=120,  # 训练迭代次数
        EPOCH=1,  # 迭代周期
        SHOTS=1024  # 解码答案时的采样数
    )

    # 打印训练的最优结果
    print("最优解为：", mbqc_result[0])
    print("最优值为： ", mbqc_result[1])

if __name__ == '__main__':
    main()

至此，我们完成了本教程的两个完整示例。MB-QAOA 算法实现预示着 MBQC 模型在量子机器学习领域强大的潜力。很显然，MBQC 模型所能够处理的算法不仅有 QAOA，我们非常期待 MBQC 这种非同寻常的计算方式能在某些特殊情境下展现出无与伦比的优势！

---

## 参考文献

[1] Farhi, Edward, et al. "A quantum approximate optimization algorithm." [arXiv preprint arXiv:1411.4028 (2014).](https://arxiv.org/abs/1411.4028)